In [1]:
import binascii
from collections import Counter

In [2]:
#from https://joernhees.de/blog/2010/09/21/how-to-convert-hex-strings-to-binary-ascii-strings-in-python-incl-8bit-space/
binary = lambda x: " ".join(reversed( [i+j for i,j in zip( *[ ["{0:04b}".format(int(c,16)) for c in reversed("0"+x)][n::2] for n in [1,0] ] ) ] ))

In [3]:
# Virtual addresses
hex_strings = ['0x00006200','0x00007000','0x00008000','0x00008800','0x0000a000','0x0000b000','0x0000c000','0x0000c800', '0x0000e000','0x0000f000','0x00010000','0x00010800','0x00012000','0x00013000','0x00014000','0x00014800']
bin_strings_v = []

for hex in hex_strings:
    bin_strings_v.append(binary(hex.replace('0x','')))
    
print(bin_strings_v)

['00000000 00000000 01100010 00000000', '00000000 00000000 01110000 00000000', '00000000 00000000 10000000 00000000', '00000000 00000000 10001000 00000000', '00000000 00000000 10100000 00000000', '00000000 00000000 10110000 00000000', '00000000 00000000 11000000 00000000', '00000000 00000000 11001000 00000000', '00000000 00000000 11100000 00000000', '00000000 00000000 11110000 00000000', '00000000 00000001 00000000 00000000', '00000000 00000001 00001000 00000000', '00000000 00000001 00100000 00000000', '00000000 00000001 00110000 00000000', '00000000 00000001 01000000 00000000', '00000000 00000001 01001000 00000000']


In [4]:
# Physical addresses
hex_strings = ['0x55aa2ec87200','0x55aa2ec88000','0x55aa2ec89000','0x55aa2ec89800','0x55aa2ec8b000','0x55aa2ec8c000','0x55aa2ec8d000','0x55aa2ec8d800', '0x55aa2ec8f000','0x55aa2ec90000','0x55aa2ec91000','0x55aa2ec91800','0x55aa2ec93000','0x55aa2ec94000','0x55aa2ec95000','0x55aa2ec95800']
bin_strings_p = []
for hex in hex_strings:
    bin_strings_p.append(binary(hex.replace('0x','')))
    
print(bin_strings_p)

['01010101 10101010 00101110 11001000 01110010 00000000', '01010101 10101010 00101110 11001000 10000000 00000000', '01010101 10101010 00101110 11001000 10010000 00000000', '01010101 10101010 00101110 11001000 10011000 00000000', '01010101 10101010 00101110 11001000 10110000 00000000', '01010101 10101010 00101110 11001000 11000000 00000000', '01010101 10101010 00101110 11001000 11010000 00000000', '01010101 10101010 00101110 11001000 11011000 00000000', '01010101 10101010 00101110 11001000 11110000 00000000', '01010101 10101010 00101110 11001001 00000000 00000000', '01010101 10101010 00101110 11001001 00010000 00000000', '01010101 10101010 00101110 11001001 00011000 00000000', '01010101 10101010 00101110 11001001 00110000 00000000', '01010101 10101010 00101110 11001001 01000000 00000000', '01010101 10101010 00101110 11001001 01010000 00000000', '01010101 10101010 00101110 11001001 01011000 00000000']


In [5]:
def remove_space(strings):
    new_list = []
    for s in strings:
        new_list.append(s.replace(' ',''))
    return new_list

In [6]:
virtual_addresses = remove_space(bin_strings_v)
physical_addresses = remove_space(bin_strings_p)

In [7]:
def XOR(string, bit1, bit2):
    b1 = string[len(string)-1-bit1]
    b2 = string[len(string)-1-bit2]
    return int(b1)^int(b2)

In [8]:
XOR(physical_addresses[0], 9, 12)

0

In [9]:
def XOR_HASH(string, start, end):
    length = int((start-end+1)/2)
    xors = []
    for i in range(length):
        xors.append(XOR(string, start-i, start-length-i))
    return xors

In [10]:
XOR_HASH(physical_addresses[0], 23, 8)

[1, 0, 1, 1, 1, 0, 1, 0]

In [11]:
def XOR7(string):
    return XOR_HASH(string, 26, 12)

def XOR8(string):
    return XOR_HASH(string, 28, 12)

In [12]:
print("XOR-7 on eviction set")
xor7s = []
for address in virtual_addresses:
    xor7s.append(XOR7(address))
    #print(xor7s[len(xor7s)-1])
print(Counter([tuple(i) for i in xor7s]))

print('')
print("XOR-8 on eviction set")
xor8s = []
for address in virtual_addresses:
    xor8s.append(XOR8(address))
    #print(xor8s[len(xor8s)-1])
print(Counter([tuple(i) for i in xor8s]))

XOR-7 on eviction set
Counter({(0, 0, 0, 0, 0, 1, 1): 2, (0, 0, 0, 0, 1, 0, 0): 2, (0, 0, 0, 0, 1, 0, 1): 2, (0, 0, 0, 0, 1, 1, 0): 2, (0, 0, 0, 0, 1, 1, 1): 2, (0, 0, 0, 1, 0, 0, 0): 2, (0, 0, 0, 1, 0, 0, 1): 2, (0, 0, 0, 1, 0, 1, 0): 2})

XOR-8 on eviction set
Counter({(0, 0, 0, 0, 0, 0, 1, 1): 2, (0, 0, 0, 0, 0, 1, 0, 0): 2, (0, 0, 0, 0, 0, 1, 0, 1): 2, (0, 0, 0, 0, 0, 1, 1, 0): 2, (0, 0, 0, 0, 0, 1, 1, 1): 2, (0, 0, 0, 0, 1, 0, 0, 0): 2, (0, 0, 0, 0, 1, 0, 0, 1): 2, (0, 0, 0, 0, 1, 0, 1, 0): 2})


In [26]:
# More uniformely distributed results: XOR-ing the bits that change between our physical addresses
results = []

for address in physical_addresses:
    results.append(XOR_HASH(address, 34, 14))
    print(results[len(results)-1])
    
print(Counter([tuple(i) for i in results])) # Check repetitions

[0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
Counter({(0, 0, 1, 0, 0, 0, 0, 1, 1, 0): 8, (0, 0, 1, 0, 0, 0, 0, 1, 0, 1): 7, (0, 0, 1, 0, 0, 0, 0, 1, 1, 1): 1})
